In [1]:
# import modules
import torch    

In [2]:
import numpy as np
import json
import copy
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.nn.utils.rnn import pad_sequence
import os
import csv
from modules_modified import ISAB, SAB, PMA
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [3]:
# import models
from model import (
    SetTransformerClassifierXY,
    SetTransformerClassifierXYAdditive,
    SetTransformerClassifier,
    DeepSetClassifierXYAdditive,
    DeepSetClassifierXY,
    DeepSetClassifier,
    SetTransformerOrdinalXY,
    SetTransformerOrdinalXYAdditive,
    SetTransformerOrdinal,
    DeepSetOrdinalXYAdditive,
    DeepSetOrdinalXY,
    DeepSetOrdinal,
    SoftVotingEnsemble,
    GeometricMeanEnsemble,
    MedianEnsemble,
    TrimmedMeanEnsemble,
    StackingEnsemble,
    AdaBoostEnsemble,
    GBMEnsemble,
    XGBoostEnsemble,
    LightGBMEnsemble,
)
from utils_ordinal import ordinal_logistic_loss, cumulative_to_labels, threshold_accuracy


In [4]:
# Mappings --------------------------------------------------------
# Map each hold like "A1"…"K18" to an integer 0…(11*18−1)=197
cols = [chr(c) for c in range(ord('A'), ord('K')+1)]
rows = list(range(1, 19))
hold_to_idx = {f"{c}{r}": i for i, (c, r) in enumerate((c, r) for r in rows for c in cols)}


# Map grades "V4"…"V11" 
grade_to_label = {f"V{i}": i - 4 for i in range(4, 12)}  
label_to_grade = {v: k for k, v in grade_to_label.items()}
print(hold_to_idx)

{'A1': 0, 'B1': 1, 'C1': 2, 'D1': 3, 'E1': 4, 'F1': 5, 'G1': 6, 'H1': 7, 'I1': 8, 'J1': 9, 'K1': 10, 'A2': 11, 'B2': 12, 'C2': 13, 'D2': 14, 'E2': 15, 'F2': 16, 'G2': 17, 'H2': 18, 'I2': 19, 'J2': 20, 'K2': 21, 'A3': 22, 'B3': 23, 'C3': 24, 'D3': 25, 'E3': 26, 'F3': 27, 'G3': 28, 'H3': 29, 'I3': 30, 'J3': 31, 'K3': 32, 'A4': 33, 'B4': 34, 'C4': 35, 'D4': 36, 'E4': 37, 'F4': 38, 'G4': 39, 'H4': 40, 'I4': 41, 'J4': 42, 'K4': 43, 'A5': 44, 'B5': 45, 'C5': 46, 'D5': 47, 'E5': 48, 'F5': 49, 'G5': 50, 'H5': 51, 'I5': 52, 'J5': 53, 'K5': 54, 'A6': 55, 'B6': 56, 'C6': 57, 'D6': 58, 'E6': 59, 'F6': 60, 'G6': 61, 'H6': 62, 'I6': 63, 'J6': 64, 'K6': 65, 'A7': 66, 'B7': 67, 'C7': 68, 'D7': 69, 'E7': 70, 'F7': 71, 'G7': 72, 'H7': 73, 'I7': 74, 'J7': 75, 'K7': 76, 'A8': 77, 'B8': 78, 'C8': 79, 'D8': 80, 'E8': 81, 'F8': 82, 'G8': 83, 'H8': 84, 'I8': 85, 'J8': 86, 'K8': 87, 'A9': 88, 'B9': 89, 'C9': 90, 'D9': 91, 'E9': 92, 'F9': 93, 'G9': 94, 'H9': 95, 'I9': 96, 'J9': 97, 'K9': 98, 'A10': 99, 'B10': 1

In [5]:
# Holds difficulty data --------------------------------------------------------
hold_difficulty = {}
with open("data/hold_difficulty.txt", "r") as f:
    for line in f:
        if ":" not in line:
            continue  # skip malformed line
        hold, rest = line.strip().split(":", 1)
        parts = rest.strip().split(",")
        difficulty = int(parts[0].strip())
        types = [t.strip() for t in parts[1:]]
        hold_difficulty[hold.strip()] = (difficulty, types)
    print("successfully parsed hold difficulty file")

# prepare type vocabulary
unique_types = set()
for _, (_, types) in hold_difficulty.items():
    unique_types.update(types)

type_to_idx = {t: i for i, t in enumerate(sorted(unique_types))}
print(f"successfully prepare type vocabulary")

successfully parsed hold difficulty file
successfully prepare type vocabulary


In [6]:
# assign x,y position to each holds -------------------------------
import string

# Board columns A–K → indices 0–10
cols = list(string.ascii_uppercase[:11])  # A–K
# Rows 1–18 → indices 0–17
rows = list(range(1, 19))  # 1–18

# Generate hold_to_coord dictionary
hold_to_coord = {}

for x, col in enumerate(cols):
    for y, row in enumerate(rows):
        hold_name = f"{col}{row}"
        hold_to_coord[hold_name] = (x, y)

print("successfully created (x,y) position to each hold:")
print(hold_to_coord)

successfully created (x,y) position to each hold:
{'A1': (0, 0), 'A2': (0, 1), 'A3': (0, 2), 'A4': (0, 3), 'A5': (0, 4), 'A6': (0, 5), 'A7': (0, 6), 'A8': (0, 7), 'A9': (0, 8), 'A10': (0, 9), 'A11': (0, 10), 'A12': (0, 11), 'A13': (0, 12), 'A14': (0, 13), 'A15': (0, 14), 'A16': (0, 15), 'A17': (0, 16), 'A18': (0, 17), 'B1': (1, 0), 'B2': (1, 1), 'B3': (1, 2), 'B4': (1, 3), 'B5': (1, 4), 'B6': (1, 5), 'B7': (1, 6), 'B8': (1, 7), 'B9': (1, 8), 'B10': (1, 9), 'B11': (1, 10), 'B12': (1, 11), 'B13': (1, 12), 'B14': (1, 13), 'B15': (1, 14), 'B16': (1, 15), 'B17': (1, 16), 'B18': (1, 17), 'C1': (2, 0), 'C2': (2, 1), 'C3': (2, 2), 'C4': (2, 3), 'C5': (2, 4), 'C6': (2, 5), 'C7': (2, 6), 'C8': (2, 7), 'C9': (2, 8), 'C10': (2, 9), 'C11': (2, 10), 'C12': (2, 11), 'C13': (2, 12), 'C14': (2, 13), 'C15': (2, 14), 'C16': (2, 15), 'C17': (2, 16), 'C18': (2, 17), 'D1': (3, 0), 'D2': (3, 1), 'D3': (3, 2), 'D4': (3, 3), 'D5': (3, 4), 'D6': (3, 5), 'D7': (3, 6), 'D8': (3, 7), 'D9': (3, 8), 'D10': (3, 9), '

In [7]:
class MoonBoardDataset(Dataset):
    def __init__(self, json_path, hold_to_idx, grade_to_label, hold_difficulty, type_to_idx, hold_to_coord, max_difficulty=10):
        self.hold_to_idx = hold_to_idx
        self.grade_to_label = grade_to_label
        self.hold_difficulty = hold_difficulty
        self.type_to_idx = type_to_idx
        self.hold_to_coord = hold_to_coord
        self.max_difficulty = max_difficulty

        with open(json_path, 'r') as f:
            self.raw = [json.loads(line) for line in f]

    def __len__(self):
        return len(self.raw)

    def __getitem__(self, idx):
        item = self.raw[idx]
        holds = item['holds']

        hold_idxs = []
        diff_values = []
        type_vecs = []
        xy_coords = []

        for h in holds:
            hold_idxs.append(self.hold_to_idx[h])

            difficulty, types = self.hold_difficulty[h]
            diff_values.append(difficulty / self.max_difficulty)

            # multi-hot vector
            type_vec = torch.zeros(len(self.type_to_idx), dtype=torch.float)
            for t in types:
                if t in self.type_to_idx:
                    type_vec[self.type_to_idx[t]] = 1.0
            type_vecs.append(type_vec)

            # normalized (x, y)
            x, y = self.hold_to_coord[h]
            xy_coords.append(torch.tensor([x / 10.0, y / 17.0], dtype=torch.float))

        return {
            "indices": torch.tensor(hold_idxs, dtype=torch.long),
            "difficulty": torch.tensor(diff_values, dtype=torch.float),
            "type": torch.stack(type_vecs),       # (N, T)
            "xy": torch.stack(xy_coords)          # (N, 2)
        }, torch.tensor(self.grade_to_label[item['grade']], dtype=torch.long)


In [8]:
from torch.utils.data import WeightedRandomSampler

# --- Set Hyperparameters ---
json_path = './data/cleaned_moonboard2024_grouped.json'
embed_dim = 64
batch_size = 16
lr = 1e-4
epochs = 20
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

XY_MODELS = {
    'set_transformer_xy',
    'set_transformer_additive',
    'deepset_xy',
    'deepset_xy_additive',
    'set_transformer_ordinal_xy',
    'set_transformer_ordinal_xy_additive',
    'deepset_ordinal_xy',
    'deepset_ordinal_xy_additive',
}

ORDINAL_MODELS = {
    'set_transformer_ordinal',
    'set_transformer_ordinal_xy',
    'set_transformer_ordinal_xy_additive',
    'deepset_ordinal',
    'deepset_ordinal_xy',
    'deepset_ordinal_xy_additive',
}

# --- Collate Function Factory ---
def make_collate_fn(model_type):
    def collate_fn(batch):
        X_indices = [x['indices'] for x, _ in batch]
        X_difficulty = [x['difficulty'] for x, _ in batch]
        X_type = [x['type'] for x, _ in batch]
        y_batch = [y for _, y in batch]

        X_indices = pad_sequence(X_indices, batch_first=True)
        X_difficulty = pad_sequence(X_difficulty, batch_first=True)
        X_type = pad_sequence(X_type, batch_first=True)
        y_tensor = torch.stack(y_batch)

        if model_type in XY_MODELS:
            X_xy = [x['xy'] for x, _ in batch]
            X_xy = pad_sequence(X_xy, batch_first=True)
            return (X_indices, X_difficulty, X_type, X_xy), y_tensor
        else:
            return (X_indices,), y_tensor
    return collate_fn

# --- Dataset Loader ---
def load_dataset(json_path, hold_to_idx, grade_to_label, hold_difficulty, type_to_idx, hold_to_coord):
    return MoonBoardDataset(json_path, hold_to_idx, grade_to_label, hold_difficulty, type_to_idx, hold_to_coord)

# --- DataLoader Preparation ---
def prepare_dataloaders(dataset, grade_to_label, batch_size, collate_fn):
    targets = [grade_to_label[item['grade']] for item in dataset.raw]
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(targets), y=targets)
    class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

    train_idx, val_idx = train_test_split(
        list(range(len(dataset))), test_size=0.2, stratify=targets, random_state=42
    )

    train_data = Subset(dataset, train_idx)
    val_data = Subset(dataset, val_idx)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader, class_weights, train_idx, val_idx

# --- Training ---
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, is_ordinal=False):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for X, y in train_loader:
            inputs = tuple(x.to(device) for x in X)
            y = y.to(device)
            payload = inputs[0] if len(inputs) == 1 else inputs
            outputs = model(payload)
            if is_ordinal:
                probs, logits = outputs
                loss = criterion(logits, y)
            else:
                logits = outputs
                loss = criterion(logits, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch:02d} — loss: {total_loss / len(train_loader):.4f}")
    return model

# --- Main Per Model ---
def main(model_type):
    dataset = load_dataset(json_path, hold_to_idx, grade_to_label, hold_difficulty, type_to_idx, hold_to_coord)
    targets = [grade_to_label[item['grade']] for item in dataset.raw]
    num_classes = len(np.unique(targets))
    vocab_size = len(hold_to_idx)
    type_vec_dim = len(type_to_idx)
    is_ordinal = model_type in ORDINAL_MODELS

    if model_type == 'set_transformer':
        ModelClass = SetTransformerClassifier
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
    elif model_type == 'set_transformer_xy':
        ModelClass = SetTransformerClassifierXY
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'set_transformer_additive':
        ModelClass = SetTransformerClassifierXYAdditive
        kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'deepset':
        ModelClass = DeepSetClassifier
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
    elif model_type == 'deepset_xy':
        ModelClass = DeepSetClassifierXY
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'deepset_xy_additive':
        ModelClass = DeepSetClassifierXYAdditive
        kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'set_transformer_ordinal':
        ModelClass = SetTransformerOrdinal
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
    elif model_type == 'set_transformer_ordinal_xy':
        ModelClass = SetTransformerOrdinalXY
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'set_transformer_ordinal_xy_additive':
        ModelClass = SetTransformerOrdinalXYAdditive
        kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'deepset_ordinal':
        ModelClass = DeepSetOrdinal
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
    elif model_type == 'deepset_ordinal_xy':
        ModelClass = DeepSetOrdinalXY
        kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    elif model_type == 'deepset_ordinal_xy_additive':
        ModelClass = DeepSetOrdinalXYAdditive
        kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
    else:
        raise ValueError(f"Unknown model_type: {model_type}")

    collate_fn = make_collate_fn(model_type)
    train_loader, val_loader, class_weights, train_idx, val_idx = prepare_dataloaders(dataset, grade_to_label, batch_size, collate_fn)

    model = ModelClass(**kwargs).to(device)
    model.is_ordinal = is_ordinal
    model.num_classes = num_classes

    if is_ordinal:
        def criterion_fn(logits, targets):
            return ordinal_logistic_loss(logits, targets)
    else:
        criterion_fn = torch.nn.CrossEntropyLoss(weight=class_weights)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model = train_model(model, train_loader, val_loader, criterion_fn, optimizer, epochs, is_ordinal=is_ordinal)
    return train_loader, val_loader, model, dataset, train_idx, val_idx


def train_boosting_main(model_type, num_stages=5, weak_epochs=3):
    """
    Replaces the `main(mtype)` call for boosting models.
    Trains a sequential AdaBoost-style ensemble.
    
    Returns the same tuple as `main()`:
    (train_loader, val_loader, final_model, dataset, train_idx, val_idx)
    """
    print(f"===== Training Boosting Ensemble ({model_type}, {num_stages} stages) =====")
    
    # --- 1. Standard Dataset Setup (copied from `main`) ---
    dataset = load_dataset(json_path, hold_to_idx, grade_to_label, hold_difficulty, type_to_idx, hold_to_coord)
    targets = [grade_to_label[item['grade']] for item in dataset.raw]
    num_classes = len(np.unique(targets))
    vocab_size = len(hold_to_idx)
    type_vec_dim = len(type_to_idx)
    is_ordinal = False # Boosting classifiers, not ordinal models

    collate_fn = make_collate_fn(model_type) # Use collate fn for the base model
    
    # Get train/val split (we need the indices)
    train_idx, val_idx = train_test_split(
        list(range(len(dataset))), test_size=0.2, stratify=targets, random_state=42
    )
    
    train_data_subset = Subset(dataset, train_idx)
    val_data_subset = Subset(dataset, val_idx)
    
    # This is the standard val_loader, used for final eval
    val_loader = DataLoader(val_data_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    
    # This loader is for *evaluating* the weak learner on train data (no shuffle)
    train_eval_loader = DataLoader(train_data_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # --- 2. Boosting-Specific Setup ---
    num_train_samples = len(train_idx)
    # Initialize uniform sample weights
    sample_weights = torch.full((num_train_samples,), 1.0 / num_train_samples, device=device)
    
    trained_models_list = [] # (name, model)
    model_alphas = []        # [alpha]
    
    # --- 3. The Sequential Training Loop ---
    for m in range(num_stages):
        print(f"--- Boosting Stage {m+1}/{num_stages} ---")
        
        # a. Create a new dataloader for this stage
        #    It samples from train_data_subset based on the *current* sample_weights
        sampler = WeightedRandomSampler(sample_weights.cpu(), num_train_samples, replacement=True)
        train_loader_stage = DataLoader(train_data_subset, batch_size=batch_size, sampler=sampler, collate_fn=collate_fn)

        # b. Create and train the weak learner
        #    We re-use the logic from your `main()` function to get the model
        if model_type == 'set_transformer':
            ModelClass = SetTransformerClassifier
            kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
        elif model_type == 'set_transformer_xy':
            ModelClass = SetTransformerClassifierXY
            kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
        elif model_type == 'set_transformer_additive':
            ModelClass = SetTransformerClassifierXYAdditive
            kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
        elif model_type == 'deepset':
            ModelClass = DeepSetClassifier
            kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes)
        elif model_type == 'deepset_xy':
            ModelClass = DeepSetClassifierXY
            kwargs = dict(vocab_size=vocab_size, dim_in=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
        elif model_type == 'deepset_xy_additive':
            ModelClass = DeepSetClassifierXYAdditive
            kwargs = dict(vocab_size=vocab_size, feat_dim=embed_dim, num_classes=num_classes, type_vec_dim=type_vec_dim)
        else:
            # This is the correct error message
            raise ValueError(f"Unsupported weak learner type for boosting: {model_type}")

        model_m = ModelClass(**kwargs).to(device)
        
        # We can use standard CrossEntropyLoss because the *sampler* already weighted the data
        criterion_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model_m.parameters(), lr=lr)
        
        # c. Train this weak learner (RE-USING YOUR EXISTING `train_model` FUNCTION!)
        print(f"Training weak learner {m+1} for {weak_epochs} epochs...")
        model_m = train_model(model_m, train_loader_stage, val_loader, criterion_fn, optimizer, weak_epochs, is_ordinal=is_ordinal)
        
        # d. Evaluate on *all* training data (unshuffled)
        model_m.eval()
        all_preds = []
        all_targets = []
        with torch.no_grad():
            for X, y in train_eval_loader: # Use NON-shuffled loader
                inputs = tuple(x.to(device) for x in X)
                payload = inputs[0] if len(inputs) == 1 else inputs
                logits = model_m(payload)
                all_preds.append(logits.argmax(dim=1))
                all_targets.append(y.to(device))
        
        all_preds = torch.cat(all_preds)
        all_targets = torch.cat(all_targets)
        
        # e. Compute weighted error
        is_incorrect = (all_preds != all_targets).float() # [num_train_samples]
        err_m = (is_incorrect * sample_weights).sum() # Sum of weights of incorrect samples
        
        if err_m <= 0 or err_m >= (1.0 - 1.0 / num_classes):
            print(f"Stage {m+1} model is perfect or too weak (err={err_m:.4f}). Stopping.")
            if err_m <= 0: # Add perfect model and break
                model_alphas.append(1.0) # Use a reasonable weight
                trained_models_list.append((f"boost_model_{m}", model_m))
            break
        
        # f. Compute model weight (alpha)
        alpha_m = torch.log((1.0 - err_m) / err_m) + torch.log(torch.tensor(num_classes - 1.0, device=device))
        
        # g. Update sample weights
        sample_weights *= torch.exp(alpha_m * is_incorrect)
        sample_weights /= sample_weights.sum() # Normalize
        
        # h. Save
        trained_models_list.append((f"boost_model_{m}", model_m))
        model_alphas.append(alpha_m.item())
        print(f"Stage {m+1}: Error={err_m:.4f}, Alpha={alpha_m:.4f}")
    
    # --- 4. Build the Final Ensemble Model ---
    if not trained_models_list:
        raise RuntimeError("Boosting training failed, no models were trained.")
        
    print(f"Building final ensemble with {len(trained_models_list)} models.")
    
    # *** USE THE NEW NAME HERE ***
    final_ensemble = AdaBoostEnsemble(trained_models_list, weights=model_alphas, freeze_members=True).to(device)
    
    final_ensemble.is_ordinal = is_ordinal
    final_ensemble.num_classes = num_classes
    
    # Return the same "package" as main()
    # `train_eval_loader` is the unshuffled train loader, which `compare_models`
    # can use for ensemble training (like stacking) if needed.
    return train_eval_loader, val_loader, final_ensemble, dataset, train_idx, val_idx

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as XLImage

BASE_MODEL_TYPES = [
    "set_transformer",
    "deepset",
    "set_transformer_xy",
    "deepset_xy",
    "set_transformer_additive",
    "deepset_xy_additive"
]

BOOSTING_TYPES = {
    "adaboost_deepset": "deepset",
    "adaboost_deepset_xy_additive": "deepset_xy_additive",
    "adaboost_deepset_xy": "deepset_xy",
    "adaboost_set_transformer": "set_transformer",
    "adaboost_set_transformer_additive": "set_transformer_additive",
    "adaboost_set_transformer_xy": "set_transformer_xy",
}

MODEL_TYPES = BASE_MODEL_TYPES + list(BOOSTING_TYPES.keys())

ENSEMBLE_TYPES = [
    "soft_voting_ensemble",
    "geometric_mean_ensemble",
    "median_ensemble",
    "trimmed_mean_ensemble",
    "stacking_ensemble",
    "gbm_ensemble",
    "xgboost_ensemble",
    "lightgbm_ensemble"
]

MODEL_COUNT_COLUMNS = {name: f"{name}_count" for name in MODEL_TYPES + ENSEMBLE_TYPES}


# --- plot confusion matrix and save to excel---
def save_confusion_matrix_to_excel(y_true, y_pred, class_labels, model_name, excel_path):
    # Plot confusion matrix and save as image
    cm = confusion_matrix(y_true, y_pred, labels=range(len(class_labels)), normalize='true')
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.title(f"Confusion Matrix: {model_name}")
    plt.xlabel("Predicted Grade")
    plt.ylabel("Actual Grade")
    plt.tight_layout()
    img_path = f"result/confusion_{model_name}.png"
    plt.savefig(img_path)
    plt.close()

    # Insert image into Excel (new sheet per model)
    wb = load_workbook(excel_path)
    if model_name in wb.sheetnames:
        ws = wb[model_name]
    else:
        ws = wb.create_sheet(title=model_name)
    img = XLImage(img_path)
    ws.add_image(img, "A1")
    wb.save(excel_path)
    print(f"Confusion matrix for {model_name} saved and inserted into {excel_path} (sheet: {model_name})")


# --- export the predictions to excel ---
def _update_outlier_excel(df_all_preds, model_name, outlier_filename="result/outlier.xlsx", sheet_name="outliers", threshold=3):
    """
    From a DataFrame with columns [problem_name, y_true, y_pred, diff],
    keep rows where abs(diff) > threshold and aggregate per problem_name:
        - count = number of times flagged
        - per-model counts = number of times flagged per model across runs
        - y_true = mode (most frequent true label)
        - y_pred_avg = average predicted label across occurrences
    Save to outlier.xlsx.
    """
    current_model_col = MODEL_COUNT_COLUMNS.get(model_name, f"{model_name}_count")
    model_count_columns = dict(MODEL_COUNT_COLUMNS)
    if model_name not in MODEL_COUNT_COLUMNS:
        model_count_columns[model_name] = current_model_col

    # Filter outliers
    outliers = df_all_preds.loc[df_all_preds["diff"].abs() > threshold,
                                ["problem_name", "y_true", "y_pred"]]
    if outliers.empty:
        print(f"No outliers (abs(diff) > {threshold}). Skipped creating outlier.xlsx.")
        return

    # Group & aggregate
    grouped = (outliers
               .groupby("problem_name")
               .agg(
                   count=("problem_name", "size"),
                   y_true=("y_true", lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0]),
                   y_pred_avg=("y_pred", lambda x: round(pd.to_numeric(x, errors="coerce").mean(), 2))
               )
               .reset_index())

    for col in model_count_columns.values():
        if col not in grouped.columns:
            grouped[col] = 0
    grouped[current_model_col] = grouped["count"]

    # If a previous file exists, merge and accumulate counts
    if os.path.exists(outlier_filename):
        try:
            existing = pd.read_excel(outlier_filename, sheet_name=sheet_name)
            for col in model_count_columns.values():
                if col not in existing.columns:
                    existing[col] = 0
            if set(existing.columns) >= {"problem_name", "count", "y_true", "y_pred_avg"}:
                merged = pd.concat([existing, grouped], ignore_index=True, sort=False)
                for col in model_count_columns.values():
                    if col not in merged.columns:
                        merged[col] = 0
                agg_map = {
                    "count": ("count", "sum"),
                    "y_true": ("y_true", lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0]),
                    "y_pred_avg": ("y_pred_avg", "mean")
                }
                agg_map.update({col: (col, "sum") for col in model_count_columns.values()})
                grouped = (merged
                           .groupby("problem_name")
                           .agg(**agg_map)
                           .reset_index())
            # else keep grouped as new
        except Exception:
            pass

    grouped["y_pred_avg"] = pd.to_numeric(grouped["y_pred_avg"], errors="coerce").round(2)
    grouped["count"] = grouped["count"].fillna(0).astype(int)
    for col in model_count_columns.values():
        if col in grouped.columns:
            grouped[col] = grouped[col].fillna(0).astype(int)

    ordered_cols = ["problem_name", "count"]
    ordered_cols.extend([model_count_columns[name] for name in MODEL_TYPES + ENSEMBLE_TYPES if model_count_columns[name] in grouped.columns])
    if current_model_col in grouped.columns and current_model_col not in ordered_cols:
        ordered_cols.append(current_model_col)
    ordered_cols.extend(["y_true", "y_pred_avg"])
    ordered_cols.extend([col for col in grouped.columns if col not in ordered_cols])
    grouped = grouped[ordered_cols]

    # Save
    with pd.ExcelWriter(outlier_filename, engine="openpyxl", mode="w") as writer:
        grouped.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Outliers saved to: {os.path.abspath(outlier_filename)}")


def export_predictions_to_excel(model, dataloader, device, grade_to_label, excel_path, sheet_name, model_name=None):
    results = []
    raw_dataset = dataloader.dataset.dataset  # MoonBoardDataset
    indices = dataloader.dataset.indices      # Subset indices
    label_to_grade = {v: k for k, v in grade_to_label.items()}
    current_index = 0

    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            if isinstance(X, tuple):
                inputs = tuple(x.to(device) for x in X)
                payload = inputs[0] if len(inputs) == 1 else inputs
            else:
                payload = X.to(device)
            outputs = model(payload)
            if isinstance(outputs, tuple):
                if getattr(model, "is_ordinal", False):
                    probs = outputs[0]
                    preds_tensor = cumulative_to_labels(probs)
                else:
                    probs = outputs[0]
                    preds_tensor = probs.argmax(dim=1)
            else:
                preds_tensor = outputs.argmax(dim=1)
            y = y.to(device)
            preds_cpu = preds_tensor.detach().cpu()
            y_cpu = y.detach().cpu()
            for i in range(y_cpu.size(0)):
                real_label = int(y_cpu[i].item())
                pred_label = int(preds_cpu[i].item())
                dataset_index = indices[current_index]
                current_index += 1
                raw_item = raw_dataset.raw[dataset_index]
                problem_name = raw_item.get('problem_name', f"problem_{dataset_index}")
                results.append({
                    "problem_name": problem_name,
                    "y_true": real_label,  # keep numeric for averaging/aggregation
                    "y_pred": pred_label,
                    "diff": real_label - pred_label
                })

    df = pd.DataFrame(results)

    if model_name is None:
        model_name = sheet_name
    df["model_name"] = model_name
    df["model"] = model_name

    # 1) Save all predictions into your main Excel file
    with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        # Convert numeric labels back to grade strings for readability
        df_out = df.copy()
        df_out["y_true"] = df_out["y_true"].map(lambda x: label_to_grade.get(x, f"Unknown({x})"))
        df_out["y_pred"] = df_out["y_pred"].map(lambda x: label_to_grade.get(x, f"Unknown({x})"))
        df_out.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Predictions for {sheet_name} exported to: {excel_path}")

    # 2) Create/update outlier.xlsx (problem_name, count, per-model counts, y_true, y_pred_avg)
    _update_outlier_excel(df, model_name=model_name, outlier_filename="result/outlier.xlsx", sheet_name="outliers", threshold=3)


# --- compute training and validation accuracy ---
def compute_accuracy(model, dataloader, device):
    strict_correct, loose_correct, total = 0, 0, 0
    y_true, y_pred = [], []
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            X = tuple(x.to(device) for x in X)
            y = y.to(device)

            payload = X[0] if len(X) == 1 else X
            outputs = model(payload)

            if isinstance(outputs, tuple):
                if getattr(model, "is_ordinal", False):
                    probs = outputs[0]
                    preds_tensor = cumulative_to_labels(probs)
                else:
                    probs = outputs[0]
                    preds_tensor = probs.argmax(dim=1)
            else:
                preds_tensor = outputs.argmax(dim=1)

            if isinstance(preds_tensor, torch.Tensor):
                preds_tensor = preds_tensor.to(y.device)
            preds = preds_tensor
            total += y.size(0)
            strict_correct += (preds == y).sum().item()
            loose_correct += ((preds - y).abs() <= 1).sum().item()
            y_true.extend(y.cpu().numpy())
            y_pred.extend(preds.detach().cpu().numpy())
    strict_acc = 100.0 * strict_correct / total
    loose_acc = 100.0 * loose_correct / total
    return strict_acc, loose_acc, y_true, y_pred


def log_accuracy_to_csv(model_type, train_strict_acc, train_loose_acc, val_strict_acc, val_loose_acc, csv_path="result/accuracy.csv"):
    file_exists = os.path.isfile(csv_path)
    with open(csv_path, mode='a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow([
                "model",
                "Train Strict Accuracy (%)",
                "Train ±1 Grade Accuracy (%)",
                "Val Strict Accuracy (%)",
                "Val ±1 Grade Accuracy (%)"
            ])
        writer.writerow([
            model_type,
            round(train_strict_acc, 2),
            round(train_loose_acc, 2),
            round(val_strict_acc, 2),
            round(val_loose_acc, 2)
        ])

In [10]:
def train_stacking_meta_model(stacking_model, dataloader, device, epochs=5, lr=1e-3):
    """Train stacking meta-learner on frozen base model outputs."""
    if epochs <= 0:
        return
    stacking_model.meta_model.train()
    for member in stacking_model.models.values():
        member.eval()
    optimizer = torch.optim.Adam(stacking_model.meta_model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        epoch_loss = 0.0
        num_samples = 0
        for X, y in dataloader:
            inputs = tuple(x.to(device) for x in X)
            targets = y.to(device)
            member_feats = stacking_model._member_features(inputs)
            M, B, C = member_feats.shape
            if stacking_model.combine == "mean":
                feat = member_feats.mean(dim=0)
            else:
                feat = member_feats.permute(1, 0, 2).reshape(B, M * C)

            logits = stacking_model.meta_model(feat)
            loss = criterion(logits, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_size = targets.size(0)
            epoch_loss += loss.item() * batch_size
            num_samples += batch_size

        denom = num_samples if num_samples > 0 else 1
        avg_loss = epoch_loss / denom
        print(f"Stacking meta epoch {epoch + 1}: loss={avg_loss:.4f}")

    stacking_model.meta_model.eval()


def train_tree_meta_model(tree_ensemble, dataloader, device):
    """Fit a tree-based meta-learner (GBM/XGBoost/LightGBM) on frozen base outputs."""
    tree_ensemble.eval()
    for member in tree_ensemble.models.values():
        member.eval()
    feature_blocks = []
    target_blocks = []
    for X, y in dataloader:
        inputs = tuple(x.to(device) for x in X)
        member_feats = tree_ensemble._member_features(inputs)
        feat = tree_ensemble._build_feature_matrix(member_feats)
        feature_blocks.append(feat.detach().cpu().numpy())
        target_blocks.append(y.detach().cpu().numpy())
    if not feature_blocks:
        raise RuntimeError("No data available to fit tree-based meta learner.")
    features = np.concatenate(feature_blocks, axis=0)
    targets = np.concatenate(target_blocks, axis=0)
    tree_ensemble.fit_meta_model(features, targets)


def build_ensemble_models(
    ensemble_names,
    base_model_items,
    ensemble_weights,
    num_classes,
    device,
    train_loader,
    stacking_meta_epochs=5,
    stacking_meta_lr=1e-3,
    label_suffix="",
):
    """Create configured ensemble models from trained base models."""
    ensembles = {}
    base_items = list(base_model_items)

    if not base_items:
        return ensembles

    def _resolve_group_weights(items):
        if ensemble_weights is None:
            return None
        if isinstance(ensemble_weights, dict):
            filtered = {name: ensemble_weights[name] for name, _ in items if name in ensemble_weights}
            if len(filtered) != len(items):
                missing = [name for name, _ in items if name not in filtered]
                if missing:
                    print(f"Warning: missing weights for {missing}; using uniform weights.")
                return None
            return filtered
        weight_list = list(ensemble_weights)
        if len(weight_list) != len(items):
            print("Warning: weight list length mismatch; using uniform weights.")
            return None
        return weight_list

    resolved_weights = _resolve_group_weights(base_items)

    for base_name in ensemble_names:
        cloned_items = [(model_name, copy.deepcopy(model)) for model_name, model in base_items]
        if not cloned_items:
            continue

        weights = resolved_weights
        if isinstance(weights, list):
            weights = list(weights)

        ensemble_key = f"{base_name}{label_suffix}" if label_suffix else base_name

        if base_name == "soft_voting_ensemble":
            ensemble_model = SoftVotingEnsemble(cloned_items, weights=weights, freeze_members=True).to(device)
        elif base_name == "geometric_mean_ensemble":
            ensemble_model = GeometricMeanEnsemble(cloned_items, weights=weights, freeze_members=True).to(device)
        elif base_name == "median_ensemble":
            ensemble_model = MedianEnsemble(cloned_items, weights=weights, freeze_members=True).to(device)
        elif base_name == "trimmed_mean_ensemble":
            ensemble_model = TrimmedMeanEnsemble(cloned_items, weights=weights, freeze_members=True, trim_frac=0.2).to(device)
        elif base_name == "stacking_ensemble":
            feature_dim = len(cloned_items) * num_classes
            meta_model = nn.Linear(feature_dim, num_classes).to(device)
            ensemble_model = StackingEnsemble(
                cloned_items,
                weights=weights,
                freeze_members=True,
                meta_model=meta_model,
                feature_source="logits",
                combine="concat",
            ).to(device)
            train_stacking_meta_model(
                ensemble_model,
                train_loader,
                device,
                epochs=stacking_meta_epochs,
                lr=stacking_meta_lr,
            )
        elif base_name == "gbm_ensemble":
            try:
                ensemble_model = GBMEnsemble(
                    cloned_items,
                    weights=weights,
                    freeze_members=True,
                    num_classes=num_classes,
                    feature_source="logits",
                    combine="concat",
                    meta_kwargs={"random_state": 42},
                ).to(device)
            except ImportError as exc:
                print(f"Skipping {base_name}: {exc}")
                continue
            train_tree_meta_model(ensemble_model, train_loader, device)
        elif base_name == "xgboost_ensemble":
            try:
                ensemble_model = XGBoostEnsemble(
                    cloned_items,
                    weights=weights,
                    freeze_members=True,
                    num_classes=num_classes,
                    feature_source="logits",
                    combine="concat",
                    meta_kwargs={"n_estimators": 300, "learning_rate": 0.05, "max_depth": 4},
                ).to(device)
            except ImportError as exc:
                print(f"Skipping {base_name}: {exc}")
                continue
            train_tree_meta_model(ensemble_model, train_loader, device)
        elif base_name == "lightgbm_ensemble":
            try:
                ensemble_model = LightGBMEnsemble(
                    cloned_items,
                    weights=weights,
                    freeze_members=True,
                    num_classes=num_classes,
                    feature_source="logits",
                    combine="concat",
                    meta_kwargs={"n_estimators": 300, "learning_rate": 0.05, "max_depth": -1},
                ).to(device)
            except ImportError as exc:
                print(f"Skipping {base_name}: {exc}")
                continue
            train_tree_meta_model(ensemble_model, train_loader, device)
        else:
            print(f"Unknown ensemble type '{base_name}', skipping.")
            continue

        ensemble_model.eval()
        ensembles[ensemble_key] = ensemble_model

    return ensembles


In [11]:
def compare_models(
    model_types=None,
    include_ensemble=True,
    ensemble_weights=None,
    ensemble_types=None,
    stacking_meta_epochs=5,
    stacking_meta_lr=1e-3,
    boosting_num_stages=5, 
    boosting_weak_epochs=3 
):
    model_types = model_types or MODEL_TYPES
    results = []
    excel_path = "result/model_comparison_results.xlsx"
    class_labels = [f"V{i}" for i in range(4, 12)]

    # BOOSTING_TYPES (defined globally) maps each boosting variant to its base learner.

    trained_models = {}
    base_dataset = None
    base_train_idx = None
    base_val_idx = None
    num_classes = None

    # --- THIS IS THE MODIFIED LOOP ---
    for idx, mtype in enumerate(model_types):
        print(f"===== Processing {mtype} =====")
        
        if mtype in BOOSTING_TYPES:
            # --- This is a SEQUENTIAL (Boosting) Model ---
            base_model_type = BOOSTING_TYPES[mtype]
            # Call our new boosting trainer
            train_loader, val_loader, model, dataset, train_idx, val_idx = train_boosting_main(
                model_type=base_model_type,
                num_stages=boosting_num_stages,
                weak_epochs=boosting_weak_epochs
            )
        else:
            # --- This is a PARALLEL (Standard) Model ---
            print(f"===== Training {mtype} =====")
            train_loader, val_loader, model, dataset, train_idx, val_idx = main(mtype)

        # --- THE REST OF THE LOOP IS UNCHANGED ---
        # Because both `main` and `train_boosting_main` return the same
        # tuple, the evaluation code works perfectly for both.
        model.eval()
        trained_models[mtype] = model
        if base_dataset is None:
            base_dataset = dataset
            base_train_idx = train_idx
            base_val_idx = val_idx
        if num_classes is None:
            num_classes = getattr(model, "num_classes", None)

        train_strict_acc, train_loose_acc, _, _ = compute_accuracy(model, train_loader, device)
        val_strict_acc, val_loose_acc, y_true, y_pred = compute_accuracy(model, val_loader, device)

        log_accuracy_to_csv(mtype, train_strict_acc, train_loose_acc, val_strict_acc, val_loose_acc)

        results.append({
            "Model Type": mtype,
            "Train Strict Accuracy (%)": round(train_strict_acc, 2),
            "Train ±1 Grade Accuracy (%)": round(train_loose_acc, 2),
            "Val Strict Accuracy (%)": round(val_strict_acc, 2),
            "Val ±1 Grade Accuracy (%)": round(val_loose_acc, 2),
        })

        if idx == 0:
            df_results = pd.DataFrame(results)
            df_results.to_excel(excel_path, index=False)

        save_confusion_matrix_to_excel(y_true, y_pred, class_labels, mtype, excel_path)
        export_predictions_to_excel(
            model,
            val_loader,
            device,
            grade_to_label,
            excel_path,
            sheet_name=f"{mtype}_preds",
            model_name=mtype,
        )

    # --- ENSEMBLE EVALUATION SECTION ---
    if include_ensemble and trained_models:
        effective_ensemble_types = ensemble_types or ENSEMBLE_TYPES
        if not effective_ensemble_types:
            print("No ensemble types specified; skipping ensemble evaluation.")
        else:
            if base_dataset is None or base_train_idx is None or base_val_idx is None:
                raise RuntimeError("Dataset indices are unavailable for ensemble evaluation.")
            if num_classes is None:
                raise RuntimeError("Unable to determine number of classes for ensembles.")

            print("===== Evaluating ensembles =====")
            collate_fn = make_collate_fn("set_transformer_xy")
            train_subset = Subset(base_dataset, base_train_idx)
            val_subset = Subset(base_dataset, base_val_idx)
            ensemble_train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
            ensemble_val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

            # Filter trained_models to EXCLUDE boosting models from being ensembled
            base_items = [(name, model) for name, model in trained_models.items() if name not in BOOSTING_TYPES]

            # Now ensemble_groups will only contain the original base models
            ensemble_groups = [
                ("all", base_items),
                ("set_transformer", [(name, model) for name, model in base_items if "set_transformer" in name]),
                ("deepset", [(name, model) for name, model in base_items if "deepset" in name]),
            ]

            for group_name, group_items in ensemble_groups:
                if not group_items:
                    print(f"Skipping {group_name} ensemble group (no models).")
                    continue

                print(f"--- Evaluating {group_name} ensembles ({len(group_items)} models) ---")
                ensembles = build_ensemble_models(
                    effective_ensemble_types,
                    group_items,
                    ensemble_weights,
                    num_classes,
                    device,
                    ensemble_train_loader,
                    stacking_meta_epochs=stacking_meta_epochs,
                    stacking_meta_lr=stacking_meta_lr,
                    label_suffix=f"_{group_name}",
                )

                if not ensembles:
                    print(f"No ensembles constructed for group {group_name}.")
                    continue

                for name, ensemble_model in ensembles.items():
                    MODEL_COUNT_COLUMNS.setdefault(name, f"{name}_count")

                    train_strict_acc, train_loose_acc, _, _ = compute_accuracy(ensemble_model, ensemble_train_loader, device)
                    val_strict_acc, val_loose_acc, y_true, y_pred = compute_accuracy(ensemble_model, ensemble_val_loader, device)

                    log_accuracy_to_csv(name, train_strict_acc, train_loose_acc, val_strict_acc, val_loose_acc)

                    results.append({
                        "Model Type": name,
                        "Train Strict Accuracy (%)": round(train_strict_acc, 2),
                        "Train ±1 Grade Accuracy (%)": round(train_loose_acc, 2),
                        "Val Strict Accuracy (%)": round(val_strict_acc, 2),
                        "Val ±1 Grade Accuracy (%)": round(val_loose_acc, 2),
                    })

                    save_confusion_matrix_to_excel(y_true, y_pred, class_labels, name, excel_path)
                    export_predictions_to_excel(
                        ensemble_model,
                        ensemble_val_loader,
                        device,
                        grade_to_label,
                        excel_path,
                        sheet_name=f"{name}_preds",
                        model_name=name,
                    )

    df_results = pd.DataFrame(results)
    with pd.ExcelWriter(excel_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
        df_results.to_excel(writer, sheet_name="Summary", index=False)
    print("=== Model Comparison Summary ===")
    print(df_results)

In [12]:
# usage
for i in range(25):
    compare_models()

===== Processing set_transformer =====
===== Training set_transformer =====
Epoch 01 — loss: 1.7663
Epoch 02 — loss: 1.6245
Epoch 03 — loss: 1.5686
Epoch 04 — loss: 1.5289
Epoch 05 — loss: 1.5012
Epoch 06 — loss: 1.4690
Epoch 07 — loss: 1.4330
Epoch 08 — loss: 1.4015
Epoch 09 — loss: 1.3684
Epoch 10 — loss: 1.3361
Epoch 11 — loss: 1.3056
Epoch 12 — loss: 1.2647
Epoch 13 — loss: 1.2437
Epoch 14 — loss: 1.2000
Epoch 15 — loss: 1.1747
Epoch 16 — loss: 1.1402
Epoch 17 — loss: 1.1012
Epoch 18 — loss: 1.0582
Epoch 19 — loss: 1.0334
Epoch 20 — loss: 1.0036
Confusion matrix for set_transformer saved and inserted into result/model_comparison_results.xlsx (sheet: set_transformer)
Predictions for set_transformer_preds exported to: result/model_comparison_results.xlsx
Outliers saved to: /Users/patrickdharma/Desktop/university/卒業課題/my_models/grade_predictor/result/outlier.xlsx
===== Processing deepset =====
===== Training deepset =====
Epoch 01 — loss: 1.8856
Epoch 02 — loss: 1.6113
Epoch 03 — loss

/Users/patrickdharma/anaconda3/envs/sousei/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Predictions for adaboost_deepset_xy_additive_preds exported to: result/model_comparison_results.xlsx
Outliers saved to: /Users/patrickdharma/Desktop/university/卒業課題/my_models/grade_predictor/result/outlier.xlsx
===== Processing adaboost_deepset_xy =====
===== Training Boosting Ensemble (deepset_xy, 5 stages) =====
--- Boosting Stage 1/5 ---
Training weak learner 1 for 3 epochs...
Epoch 01 — loss: 1.6170
Epoch 02 — loss: 1.3598
Epoch 03 — loss: 1.2931
Stage 1: Error=0.5239, Alpha=1.6959
--- Boosting Stage 2/5 ---
Training weak learner 2 for 3 epochs...
Epoch 01 — loss: 1.7758
Epoch 02 — loss: 1.5785
Epoch 03 — loss: 1.5224
Stage 2: Error=0.6277, Alpha=1.2695
--- Boosting Stage 3/5 ---
Training weak learner 3 for 3 epochs...
Epoch 01 — loss: 1.8327
Epoch 02 — loss: 1.6754
Epoch 03 — loss: 1.6236
Stage 3: Error=0.6691, Alpha=1.0875
--- Boosting Stage 4/5 ---
Training weak learner 4 for 3 epochs...
Epoch 01 — loss: 1.8692
Epoch 02 — loss: 1.7948
Epoch 03 — loss: 1.7263
Stage 4: Error=0.755

/Users/patrickdharma/anaconda3/envs/sousei/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Confusion matrix for adaboost_deepset_xy saved and inserted into result/model_comparison_results.xlsx (sheet: adaboost_deepset_xy)


...

Confusion matrix for gbm_ensemble_deepset saved and inserted into result/model_comparison_results.xlsx (sheet: gbm_ensemble_deepset)


/Users/patrickdharma/anaconda3/envs/sousei/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Predictions for gbm_ensemble_deepset_preds exported to: result/model_comparison_results.xlsx
Outliers saved to: /Users/patrickdharma/Desktop/university/卒業課題/my_models/grade_predictor/result/outlier.xlsx


/Users/patrickdharma/anaconda3/envs/sousei/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


=== Model Comparison Summary ===
                                 Model Type  Train Strict Accuracy (%)  \
0                           set_transformer                      60.70   
1                                   deepset                      42.83   
2                        set_transformer_xy                      58.51   
3                                deepset_xy                      45.60   
4                  set_transformer_additive                      55.81   
5                       deepset_xy_additive                      46.73   
6                          adaboost_deepset                      45.43   
7              adaboost_deepset_xy_additive                      45.85   
8                       adaboost_deepset_xy                      46.15   
9                  adaboost_set_transformer                      52.86   
10        adaboost_set_transformer_additive                      48.34   
11              adaboost_set_transformer_xy                      53.01   
12   

In [ ]:
# Ordinal evaluation helpers
def evaluate_ordinal_thresholds(model, loader, grade_to_label, device, decision_threshold=0.5, model_name=None, output_dir='./result'):
    model.eval()
    probs_list = []
    targets_list = []
    with torch.no_grad():
        for X, y in loader:
            inputs = tuple(x.to(device) for x in X)
            y = y.to(device)
            payload = inputs[0] if len(inputs) == 1 else inputs
            outputs = model(payload)
            if not isinstance(outputs, tuple):
                raise ValueError('Model is not configured for ordinal outputs.')
            probs, logits = outputs
            probs_list.append(probs.cpu())
            targets_list.append(y.cpu())
    if not probs_list:
        raise ValueError('No samples available for ordinal evaluation.')
    probs = torch.cat(probs_list, dim=0)
    targets = torch.cat(targets_list, dim=0)
    acc_per_threshold = threshold_accuracy(probs, targets, threshold=decision_threshold).cpu()
    grade_by_label = {v: k for k, v in grade_to_label.items()}
    threshold_labels = []
    for idx in range(acc_per_threshold.size(0)):
        grade = grade_by_label.get(idx, f'label_{idx}')
        threshold_labels.append(f"P(>{grade})")
    df = pd.DataFrame({
        'threshold': threshold_labels,
        'accuracy': (acc_per_threshold.numpy() * 100).round(2)
    })
    overall_pred = cumulative_to_labels(probs, threshold=decision_threshold)
    overall_acc = (overall_pred == targets).float().mean().item() * 100
    if model_name:
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f'ordinal_metrics_{model_name}.csv')
        df.to_csv(output_path, index=False)
        print(f'Saved threshold table to {output_path}')
    print(df)
    print(f'Overall accuracy: {overall_acc:.2f}%')
    return df, overall_acc



In [ ]:
# --- Ordinal variants sweep ---
ordinal_model_types = [
    'set_transformer_ordinal',
    'set_transformer_ordinal_xy',
    'set_transformer_ordinal_xy_additive',
    'deepset_ordinal',
    'deepset_ordinal_xy',
    'deepset_ordinal_xy_additive',
]

ordinal_tables = []
ordinal_summary = []

for model_key in ordinal_model_types:
    print(f"=== Training ordinal model: {model_key} ===")
    train_loader, val_loader, model, dataset, train_idx, val_idx = main(model_key)
    table, overall_acc = evaluate_ordinal_thresholds(
        model,
        val_loader,
        grade_to_label=grade_to_label,
        device=device,
        decision_threshold=0.5,
        model_name=model_key
    )
    table = table.copy()
    table['model'] = model_key
    table['overall_accuracy'] = overall_acc
    ordinal_tables.append(table)
    ordinal_summary.append({'model': model_key, 'overall_accuracy': overall_acc})

if ordinal_tables:
    combined = pd.concat(ordinal_tables, ignore_index=True)
    summary_df = pd.DataFrame(ordinal_summary)

    threshold_order = [f"P(>{grade})" for grade in sorted(grade_to_label.keys(), key=lambda g: grade_to_label[g])]
    pivot_df = (combined
                .pivot_table(index='model', columns='threshold', values='accuracy')
                .reindex(columns=[c for c in threshold_order if c in combined['threshold'].unique()]))
    pivot_df = pivot_df.sort_index()
    pivot_df['Overall Accuracy'] = summary_df.set_index('model')['overall_accuracy']

    combined = combined.sort_values(['model', 'threshold']).reset_index(drop=True)
    summary_df = summary_df.sort_values('model').reset_index(drop=True)

    output_path = './result/ordinal_result.xlsx'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with pd.ExcelWriter(output_path) as writer:
        pivot_df.to_excel(writer, sheet_name='threshold_matrix')
        combined.to_excel(writer, sheet_name='threshold_long', index=False)
        summary_df.to_excel(writer, sheet_name='overall', index=False)
    print(f"Saved combined ordinal results to {output_path}")
else:
    print('No ordinal results generated.')



In [ ]:
# evaluate problems
def evaluate_problems(
    model, problem_dict, hold_to_idx, hold_difficulty, type_to_idx, device,
    grade_to_label, hold_to_coord, dataset, train_idx, val_idx, model_type
):
    label_to_grade = {v: k for k, v in grade_to_label.items()}
    print("=== MoonBoard Problem Evaluation ===")

    for fallback_name, holds in problem_dict.items():
        try:
            hold_idxs = []
            diff_values = []
            type_vecs = []
            xy_coords = []

            for h in holds:
                if h not in hold_difficulty or h not in hold_to_idx or h not in hold_to_coord:
                    raise ValueError(f"[ERROR] Hold '{h}' missing from required dictionaries.")

                hold_idxs.append(hold_to_idx[h])
                difficulty, types = hold_difficulty[h]
                diff_values.append(difficulty / 10.0)

                # Multi-hot vector
                type_vec = torch.zeros(len(type_to_idx), dtype=torch.float)
                for t in types:
                    if t in type_to_idx:
                        type_vec[type_to_idx[t]] = 1.0
                type_vecs.append(type_vec)

                xy_coords.append(torch.tensor([hold_to_coord[h][0] / 10.0, hold_to_coord[h][1] / 17.0], dtype=torch.float))

            # Convert to tensors
            hold_tensor = torch.tensor(hold_idxs, dtype=torch.long).unsqueeze(0).to(device)
            difficulty_tensor = torch.tensor(diff_values, dtype=torch.float).unsqueeze(0).to(device)
            type_tensor = torch.stack(type_vecs).unsqueeze(0).to(device)
            xy_tensor = torch.stack(xy_coords).unsqueeze(0).to(device)

            model.eval()
            with torch.no_grad():
                # --- Select input format based on model_type ---
                if model_type in XY_MODELS:
                    input_data = (hold_tensor, difficulty_tensor, type_tensor, xy_tensor)
                elif model_type in {'set_transformer', 'deepset', 'set_transformer_ordinal', 'deepset_ordinal'}:
                    input_data = (hold_tensor,)
                else:
                    raise ValueError(f"Unknown model type: {model_type}")

                payload = input_data[0] if isinstance(input_data, tuple) and len(input_data) == 1 else input_data
                outputs = model(payload)
                if isinstance(outputs, tuple):
                    probs, logits = outputs
                    pred_label = (probs > 0.5).sum(dim=1).item()
                else:
                    logits = outputs
                    pred_label = logits.argmax(dim=1).item()
                pred_grade = label_to_grade.get(pred_label, f"Unknown({pred_label})")

        except Exception as e:
            print(f"[{fallback_name}] Skipping due to error: {e}")
            continue

        # Search in dataset for match
        found_idx = None
        split = "Not Found"
        setter_grade = "Unknown"
        problem_name = fallback_name

        for idx_item, item in enumerate(dataset.raw):
            if set(item['holds']) == set(holds):
                found_idx = idx_item
                setter_grade = item.get('grade', 'Unknown')
                problem_name = item.get('problem_name', fallback_name)
                if found_idx in train_idx:
                    split = "Train"
                elif found_idx in val_idx:
                    split = "Validation"
                else:
                    split = "Found (Unknown Split)"
                break

        holds_with_difficulty = {h: hold_difficulty[h][0] if h in hold_difficulty else "N/A" for h in holds}
        print(f"🔹 Problem Name   : {problem_name}")
        print(f"   Holds Used     : {holds_with_difficulty}")
        print(f"   Setter Grade   : {setter_grade}")
        print(f"   Predicted Grade: {pred_grade}")
        print(f"   Dataset Split  : {split}")

named_problems = {
    "Physical V9 Benchmark": ["I18", "J12", "F13", "D10", "E6", "J2"],
    "Triangulation V7": ["A18", "J13", "D16", "E9", "E9", "I4"],
    "warmup crimps": ["I18", "I7", "I9", "I15", "G11", "J14", "J12", "I15", "J14", "H4", "K6"],
    "Ronani V5": ["F18", "I15", "I10", "K9", "K6", "G14", "D16", "E9", "K6", "I15", "E4", "H5"],
    "Don't Fart Alan": ["K18", "J15", "F14", "F13", "D10", "E6", "I7", "I5", "F1"],
    "FINALE MAXI 2025 POCKET 2 V9": ["G3", "F3", "F4", "A6", "A11", "B17", "C9", "D17", "H18"],
    "Khai's V7": ["D18", "A15", "A12", "C9", "E7", "H8", "I6", "E1"],
    "Yums In My Tums V5": ["F18", "G12", "E1", "D13", "I9", "F8", "I2", "F16", "E4", "E6"],
}

team_problems = {
    "simma mot strommen": ["A18", "C12", "A9", "B14", "B16", "D1", "F5", "F5"],
    "MAXIMUS!": ["K18", "E3", "K14", "I13", "K7", "I2", "H16", "K11", "G7", "H4"],
    "interstate": ["K18", "H17", "J11", "I9", "G13", "H15", "I5", "I6"],
    "krakatoa pusher": ["H18", "H11", "J8", "F7", "K15", "F4", "J3"],
    "doublement": ["A18", "E16", "F8", "B14", "G8", "E12", "F4", "F3", "F3"],
    "animal instinct": ["F18", "J11", "F9", "H15", "E13", "J11", "I6", "F4"],
    "blue bin day": ["B18", "C18", "A8", "C12", "B15", "A5", "C3"]
}

# --- Classification baseline run ---
clf_train_loader, clf_val_loader, clf_model, clf_dataset, clf_train_idx, clf_val_idx = main('set_transformer_additive')

evaluate_problems(
    model=clf_model,
    problem_dict=named_problems,
    hold_to_idx=hold_to_idx,
    hold_difficulty=hold_difficulty,
    type_to_idx=type_to_idx,
    device=device,
    grade_to_label=grade_to_label,
    hold_to_coord=hold_to_coord,
    dataset=clf_dataset,
    train_idx=clf_train_idx,
    val_idx=clf_val_idx,
    model_type='set_transformer_additive'
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import torch

def visualize_attention_for_problem(model, holds, hold_to_idx, hold_difficulty, type_to_idx, hold_to_coord, device):
    model.eval()

    hold_idxs = []
    diff_values = []
    type_vecs = []
    xy_coords = []

    for h in holds:
        hold_idxs.append(hold_to_idx[h])
        difficulty, types = hold_difficulty[h]
        diff_values.append(difficulty / 10.0)

        type_vec = torch.zeros(len(type_to_idx), dtype=torch.float)
        for t in types:
            if t in type_to_idx:
                type_vec[type_to_idx[t]] = 1.0
        type_vecs.append(type_vec)

        x, y = hold_to_coord[h]
        xy_coords.append([x / 10.0, y / 17.0])

    hold_tensor = torch.tensor(hold_idxs, dtype=torch.long).unsqueeze(0).to(device)
    diff_tensor = torch.tensor(diff_values, dtype=torch.float).unsqueeze(0).to(device)
    type_tensor = torch.stack(type_vecs).unsqueeze(0).to(device)
    xy_tensor = torch.tensor(xy_coords, dtype=torch.float).unsqueeze(0).to(device)

    with torch.no_grad():
        _ = model((hold_tensor, diff_tensor, type_tensor, xy_tensor))

    attn_isab1 = model.encoder[0].mab0.attn_weights.cpu().numpy()
    attn_isab2 = model.encoder[1].mab0.attn_weights.cpu().numpy()

    num_heads = attn_isab1.shape[0]
    fig, axes = plt.subplots(2, num_heads, figsize=(4 * num_heads, 8))
    if num_heads == 1:
        axes = axes.reshape(2, 1)

    for h in range(num_heads):
        sns.heatmap(attn_isab1[h], ax=axes[0, h], cmap="viridis", xticklabels=holds)
        axes[0, h].set_title(f"ISAB1 – Head {h}")
        axes[0, h].set_xlabel("Key (Hold)")
        axes[0, h].set_ylabel("Seed")

        sns.heatmap(attn_isab2[h], ax=axes[1, h], cmap="viridis", xticklabels=holds)
        axes[1, h].set_title(f"ISAB2 – Head {h}")
        axes[1, h].set_xlabel("Key (Hold)")
        axes[1, h].set_ylabel("Seed")

    plt.tight_layout()
    plt.show()


In [ ]:
def get_avg_attention_per_hold(model, holds, hold_to_idx, hold_difficulty, type_to_idx, hold_to_coord, device):
    model.eval()

    hold_idxs = []
    diff_values = []
    type_vecs = []
    xy_coords = []

    for h in holds:
        hold_idxs.append(hold_to_idx[h])
        difficulty, types = hold_difficulty[h]
        diff_values.append(difficulty / 10.0)

        # Multi-hot type vector
        type_vec = torch.zeros(len(type_to_idx), dtype=torch.float)
        for t in types:
            if t in type_to_idx:
                type_vec[type_to_idx[t]] = 1.0
        type_vecs.append(type_vec)

        # XY coordinate
        if h not in hold_to_coord:
            raise ValueError(f"[ERROR] Hold '{h}' has no coordinate in hold_to_coord.")
        x, y = hold_to_coord[h]
        xy_coords.append([x / 10.0, y / 17.0])

    # Build model input tensors
    hold_tensor = torch.tensor(hold_idxs, dtype=torch.long).unsqueeze(0).to(device)       # (1, N)
    diff_tensor = torch.tensor(diff_values, dtype=torch.float).unsqueeze(0).to(device)    # (1, N)
    type_tensor = torch.stack(type_vecs).unsqueeze(0).to(device)                          # (1, N, T)
    xy_tensor = torch.tensor(xy_coords, dtype=torch.float).unsqueeze(0).to(device)        # (1, N, 2)

    with torch.no_grad():
        _ = model((hold_tensor, diff_tensor, type_tensor, xy_tensor))

    attn_weights = model.encoder[0].mab0.attn_weights  # shape: (heads, seeds, holds)
    avg_attn = attn_weights.mean(dim=(0, 1)).cpu().numpy()  # average across heads & seeds → (num_holds,)

    return list(zip(holds, avg_attn))


In [ ]:
# Visualize attention and scores (with XY support)

holds = named_problems["warmup crimps"]

if not hasattr(model.encoder[0], 'mab0') or not hasattr(model.encoder[0].mab0, 'attn_weights'):
    raise ValueError("The provided model does not support attention visualization.")

visualize_attention_for_problem(model, holds, hold_to_idx, hold_difficulty, type_to_idx, hold_to_coord, device)
attention_scores = get_avg_attention_per_hold(model, holds, hold_to_idx, hold_difficulty, type_to_idx, hold_to_coord, device)

# Print sorted scores
attention_scores_sorted = sorted(attention_scores, key=lambda x: x[1], reverse=True)
print("Average Attention Per Hold (sorted):")
for h, score in attention_scores_sorted:
    difficulty = hold_difficulty[h][0] if h in hold_difficulty else "N/A"
    print(f"{h}: {score:.4f} (difficulty: {difficulty})")


In [ ]:
# read accuracy.csv file
import pandas as pd

df = pd.read_csv('./result/accuracy.csv')
# print(df)

filtered_df = df[df['model'] == 'set_transformer_xy']
print(filtered_df)
